# Forecasting Walmart sales with Treeffuser

In this tutorial we show how to use Treeffuser to model and forecast Walmart sales using the M5 forecasting dataset from Kaggle.

## Getting started


To get started, we first install `treeffuser` and import the relevant libraries (if needed).

!pip install treeffuser

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from pathlib import Path

from tqdm import tqdm
from treeffuser import Treeffuser

# load autoreload extension
%load_ext autoreload
%autoreload 2

Next, create a Kaggle account and download the data from https://www.kaggle.com/competitions/m5-forecasting-accuracy/data.

If you're running this notebook in Colab, manually upload the necessary files (`calendar.csv`, `sales_train_validation.csv`, `sell_prices.csv`) to Colab by clicking the `Files` tab on the left sidebar and selecting `Upload`. Move the files into a new folder named `m5`. Once uploaded, the notebook will be able to read and process the data.

If you're running this on your local machine, you can also use Kaggle's [command-line tool](https://www.kaggle.com/docs/api) and run the following from the command line:

```bash
cd ./m5 # path to folder where you want to save the data
kaggle competitions download -c m5-forecasting-accuracy
```

Use your favorite tool to unzip the archive. In Linux/macOS,

```bash
unzip m5-forecasting-accuracy.zip
```

We'll be using the following files: `calendar.csv`, `sales_train_validation.csv`, and `sell_prices.csv`.


<!-- - `calendar.csv` - Contains information about the dates on which the products are sold.
- `sales_train_validation.csv` - Contains the historical daily unit sales data per product and store `[d_1 - d_1913]`.
- `sell_prices.csv` - Contains information about the price of the products sold per store and date. -->
<!-- - `sales_train_evaluation.csv`- Includes sales [`d_1 - d_1941]` (labels used for the Public leaderboard). -->
<!-- - `sample_submission.csv` - The correct format for submissions. Reference the Evaluation tab for more info. -->

#### Load the data

In [81]:
data_path = Path("./m5")  # change with path where you extracted the data archive

calendar_df = pd.read_csv(data_path / "calendar.csv")
sales_train_df = pd.read_csv(data_path / "sales_train_validation.csv")
sell_prices_df = pd.read_csv(data_path / "sell_prices.csv")

# add explicit columns for the day, month, year for ease of processing
calendar_df["date"] = pd.to_datetime(calendar_df["date"])
calendar_df["day"] = calendar_df["date"].dt.day
calendar_df["month"] = calendar_df["date"].dt.month
calendar_df["year"] = calendar_df["date"].dt.year

## The data

### Preprocessing
`sell_prices_df` contains the prices of each item in each store at a given time. The `wm_yr_wk` is a unique identifier for the time.

In [ ]:
sell_prices_df.head()

`calendar_df` contains information about the dates on which the products were sold.

In [ ]:
calendar_df.head()

`sales_train_df` contains the number of units sold for an item in each department and store. The sales are grouped by day: for example, the `d_1907` column has the number of units sold on the 1907-th day.

In [ ]:
sales_train_df.head()

To align the sales data with the other DataFrames, we convert `sales_train_df` to a long format. We collapse the daily sales columns `d_{i}` into a single `sales` column, with an  additional `day` column indicating the day corresponding to each sales entry.

In [ ]:
def convert_sales_data_from_wide_to_long(sales_df_wide):
    index_vars = ["item_id", "dept_id", "cat_id", "store_id", "state_id"]
    sales_df_long = pd.wide_to_long(
        sales_df_wide.iloc[:100, 1:],
        i=index_vars,
        j="day",
        stubnames=["d"],
        sep="_",
    ).reset_index()

    sales_df_long = sales_df_long.rename(columns={"d": "sales", "day": "d"})

    sales_df_long["d"] = "d_" + sales_df_long["d"].astype(
        "str"
    )  # restore "d_{i}" format for day
    return sales_df_long


sales_train_df_long = convert_sales_data_from_wide_to_long(sales_train_df)
sales_train_df_long.head()

In [ ]:
plt.hist(
    sales_train_df_long["sales"],
    bins=np.arange(0, 10 + 1.5) - 0.5,
    range=[0, 10],
    density=True,
)
plt.xticks(range(10))
plt.ylabel("Frequency of number of sales")
plt.title("Number of sales over the entire timespan")

### Train and test sets

The dataset comprises sales data of 100 items over 1,913 days. For simplicity, we select the data from the first 365 days and discard the rest.

In [ ]:
print(f"n_items = {len(sales_train_df_long['item_id'].unique())}")
print(f"n_days = {len(sales_train_df_long['d'].unique())}")

sales_train_df_long["day_number"] = sales_train_df_long["d"].str.extract("(\d+)").astype(int)
data = sales_train_df_long[sales_train_df_long["day_number"] <= 365].copy()

We compute the lags of the previous 30 days and merge the sales, calendar, and price data together.

In [ ]:
n_lags = 30

# sort data before computing lags
data_index_vars = ["item_id", "dept_id", "cat_id", "store_id", "state_id"]
data.sort_values(data_index_vars + ["day_number"], inplace=True)

for lag in range(1, n_lags + 1):
    data[f"sales_lag_{lag}"] = data.groupby(by=data_index_vars)["sales"].shift(lag)

data = data.merge(calendar_df).merge(sell_prices_df)

data.head()

Treeffuser can handle **categorical columns**, but the dtype of those columns must be set to `category` in the DataFrame.

In [89]:
categorical_columns = [
    "item_id",
    "dept_id",
    "cat_id",
    "store_id",
    "state_id",
    "d",
    "wm_yr_wk",
    "weekday",
    "event_name_1",
    "event_type_1",
    "event_name_2",
    "event_type_2",
    "snap_CA",
    "snap_TX",
    "snap_WI",
]
data[categorical_columns] = data[categorical_columns].astype("category")

Finally, for each item, we take the first 300 days as train data and use the remaining 65 data as test data for evaluation.

In [ ]:
is_train = data["day_number"] <= 300

y_name = "sales"
x_names = [
    name for name in data.columns if name != y_name and name not in ["day_number", "date"]
]

X_train, y_train, dates_train = (
    data[is_train][x_names],
    data[is_train][y_name],
    data[is_train]["date"],
)
X_test, y_test, dates_test = (
    data[~is_train][x_names],
    data[~is_train][y_name],
    data[~is_train]["date"],
)

print(X_train.shape)
print(X_test.shape)

## Probabilistic predictions with Treeffuser

We regress the sales on the following covariates.

In [ ]:
print(", ".join(map(str, X_train.columns)))

In [ ]:
model = Treeffuser(seed=0)
model.fit(X_train, y_train)

In [ ]:
y_test_samples = model.sample(X_test, n_samples=100, seed=0, n_steps=50, verbose=True)

## Newsvendor model

We illustrate the practical relevance of accurate probabilistic predictions with an application to inventory management, using the newsvendor model \citep{arrow1951optimal}. 

Assume that every day we decide how many units $q$ of an item to buy. 
We buy at a cost $c$ and sell at a price $p$. 
However, the demand $y$ is random, introducing uncertainty in our decision. 
The goal is to maximize the expected profit:
$$\max_{q} p~\mathbb{E}\left[\min(q, y)\right] - c q.$$
The optimal solution to the newsvendor problem is to buy $q = F^{-1}\left( \frac{p-c}{p} \right)$ units, where $F^{-1}$ is the quantile function of the distribution of $y$. 

Using Treeffuser, we can compute the quantiles from the samples and forecast the optimal quantity of units to buy.

To compute profits, we use the observed prices, assume a margin of $50\%$ over all products, and assume the actual number of sales of an item correspond to the demand of this item. We let Treeffuser, learn the conditional distribution of the demand of each item, estimate their quantiles, and thus determine the optimal quantity to buy. 

We use the held-out data to compute the profit made if Treeffuser was used to forecast the demand of each item and to manage the inventory of each item.

In [94]:
def newsvendor_utility(y_true, quantity_ordered, prices, stocking_cost):
    """
    The newsvendor utility function with stock q, demand y, selling price p, stocking cost c is given by
    $$ U(y, q, p, c) = p * min(y, q) - c * q $$
    """
    utility = prices * np.minimum(y_true, quantity_ordered) - stocking_cost * quantity_ordered
    return utility


def newsvendor_optimal_quantity(y_samples, prices, stocking_cost):
    """
    Returns the optimal quantity to order for the newsvendor problem.

    It is given theoeretically by:
        $$ q* = argmax_{q} E[U(y, q, p, c)] $$
    which has a closed form solution,
        $$ q* = F^{-1}( (p - c) / p) $$
    where F is the CDF of the demand distribution
    """
    # compute the target quantiles (p - c) / p
    target_quantiles = (prices - stocking_cost) / prices
    target_quantiles = np.maximum(target_quantiles, 0.0)

    # compute the empirical quantities corresponding to the target quantiles
    res = []
    for i in range(y_samples.shape[1]):
        optimal_quantities = np.quantile(y_samples[:, i], target_quantiles[i])
        res.append(optimal_quantities)
    optimal_quantities = np.array(res)
    return optimal_quantities

In [ ]:
# we don't know the profit margin of each item, so we assume it is 50%.
profit_margin = 0.5

prices = X_test["sell_price"].values
stocking_cost = prices / (1 + profit_margin)

# compute optimal quantities
optimal_quantities = newsvendor_optimal_quantity(y_test_samples, prices, stocking_cost)

# Treeffuser models continuous responses, hence we cast the predicted quantities into int
optimal_quantities = optimal_quantities.astype(int)

profit = newsvendor_utility(y_test, optimal_quantities, prices, stocking_cost)
profit.sum()

We visualize the cumulative profit, the average demand and inventory over time in the plot below.

In [ ]:
performance_data = pd.DataFrame(
    {
        "date": dates_test,
        "profit": profit,
        "demand": y_test,
        "inventory": optimal_quantities,
        "price": prices,
    }
)

# for each day, compute average demand and inventory weighted by price
daily_summary = (
    performance_data.groupby("date", group_keys=False)
    .apply(
        lambda x: pd.Series(
            {
                "profit": x["profit"].sum(),
                "avg_inventory_weighted": (x["inventory"] * x["price"]).sum()
                / x["price"].sum(),  # price-weighted average inventory
                "avg_demand_weighted": (x["demand"] * x["price"]).sum()
                / x["price"].sum(),  # price-weighted average demand
            }
        )
    )
    .sort_values("date")
    .reset_index()
)

print(daily_summary)

In [ ]:
# dictionary to store color, alpha, linewidth, and linestyle for each line
line_styles = {
    "cumulative_profit": {"color": "teal", "alpha": 1, "linewidth": 2.5, "linestyle": "-"},
    "inventory": {
        "color": "blue",
        "alpha": 0.5,
        "linewidth": 1.5,
        "linestyle": "--",
    },
    "demand": {
        "color": "orange",
        "alpha": 0.5,
        "linewidth": 1.5,
        "linestyle": "--",
    },
}

# create figure
fig, ax1 = plt.subplots(figsize=(10, 6))

# define x-axis
dates = pd.to_datetime(daily_summary["date"])

# plot cumulative profit
ax1.plot(
    dates,
    daily_summary["profit"].cumsum(),
    **line_styles["cumulative_profit"],
    label="Cumulative Profit",
)
ax1.set_xlabel("Date", fontsize=12)
ax1.set_ylabel("Cumulative Profit ($)", fontsize=12)
ax1.grid(True, linestyle="--", alpha=0.6)

# create second y-axis for price-weighted inventory and demand
ax2 = ax1.twinx()
ax2.plot(
    dates,
    daily_summary["avg_inventory_weighted"],
    **line_styles["inventory"],
    label="Avg Inventory (Price Weighted)",
)
ax2.plot(
    dates,
    daily_summary["avg_demand_weighted"],
    **line_styles["demand"],
    label="Avg Demand (Price Weighted)",
)
ax2.set_ylabel("Avg Inventory and Demand (Units)", fontsize=12)

# combine all legends into one
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(
    lines1 + lines2, labels1 + labels2, loc="upper center", bbox_to_anchor=(0.5, 1.12), ncol=3
)

fig.autofmt_xdate()  # rotate x-tick labels

plt.show()